In [2]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [3]:
api_key = "67565f211247a70ca588f9bb4e9561aa.WKluNGHOe5UR4Ot8"
llm_model = "glm-4"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_base="https://open.bigmodel.cn/api/paas/v4/",
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [4]:
# parquet files generated from indexing pipeline
INPUT_DIR = "./inputs/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [5]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

Total report count: 19
Report count after filtering by community level 2: 15


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,10,# HuaZhong University of Science and Technolog...,1,8.5,HuaZhong University of Science and Technology'...,The rating reflects the community's strong ali...,The community is structured around the Network...,[{'explanation': 'The network security class o...,"{\n ""title"": ""HuaZhong University of Scienc...",d6e25ee8-5db6-423d-ac6d-392aa8785a7f
1,11,# HuaZhong University of Science and Technolog...,1,8.0,HuaZhong University of Science and Technology'...,The rating is high due to the comprehensive ra...,The community is centered around HuaZhong Univ...,[{'explanation': 'HuaZhong University of Scien...,"{\n ""title"": ""HuaZhong University of Scienc...",d60750ba-8191-48ca-aad1-cdde6b649e77
2,12,# HuaZhong University of Science and Technolog...,1,8.5,HuaZhong University of Science and Technology'...,The rating reflects the community's strong ali...,This community is structured around the compre...,[{'explanation': 'The 课程表 provides a diverse r...,"{\n ""title"": ""HuaZhong University of Scienc...",77985104-003d-4c28-9628-7d468931b1e2
3,13,# Huazhong University of Science and Technolog...,1,8.0,Huazhong University of Science and Technology'...,The rating is high due to the specialized focu...,The community is centered around the Cybersecu...,[{'explanation': 'Classrooms B221-3 and B105 a...,"{\n ""title"": ""Huazhong University of Scienc...",4a60f2ef-bd91-4815-af15-b9c53e36d7f5
4,14,# HuaZhong University of Science and Technolog...,1,8.0,HuaZhong University of Science and Technology'...,The rating is high due to the community's clea...,The community is centered around HuaZhong Univ...,[{'explanation': 'The 网络空间安全（本硕博）202101班 class...,"{\n ""title"": ""HuaZhong University of Scienc...",b0141076-121f-46c7-9be3-0aa8b3098c7c


#### Build global context based on community reports

In [6]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [7]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [8]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [13]:
result = await search_engine.asearch(
    "介绍一下网络安全课程设计这一门课，它安排在周几第几节课上？"
)

print(result.response)

### 网络安全课程设计概述

网络安全课程设计是华中科技大学网络安全教育社区的重要组成部分，专注于网络安全领域的课程设计和实践。该课程的目标是提供学生深入理解网络安全领域的知识，以及应对行业需求的实用技能。[Data: Reports (17, 6, 4, 13, 5)]

### 课程安排

遗憾的是，目前提供的分析报告中并没有包含该课程具体的上课时间信息，如周几第几节课。为了给您提供这一具体信息，我需要查阅更多相关数据记录。但是，根据现有信息，我可以确认该课程强调实践和理论知识在网络安全领域的应用。

如果您需要了解该课程的具体上课时间，建议直接咨询华中科技大学的相关部门或查阅最新的课程安排公告。

### 结语

网络安全课程设计作为一门实践与理论相结合的课程，对于希望在网络安全领域内发展的学生来说，具有重要的意义。虽然我无法提供具体的上课时间，但希望以上信息对您有所帮助。如果您有其他问题或需要更多信息，请随时告知。


In [14]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,17,HuaZhong University's Cybersecurity Educationa...,1.000000,# HuaZhong University's Cybersecurity Educatio...,8.0
1,6,Huazhong University of Science and Technology'...,1.000000,# Huazhong University of Science and Technolog...,8.0
2,4,HuaZhong University of Science and Technology'...,0.944444,# HuaZhong University of Science and Technolog...,8.0
3,13,Huazhong University of Science and Technology'...,0.888889,# Huazhong University of Science and Technolog...,8.0
4,5,HuaZhong University of Science and Technology'...,0.888889,# HuaZhong University of Science and Technolog...,7.5
5,16,HuaZhong University of Science and Technology'...,0.833333,# HuaZhong University of Science and Technolog...,8.0
6,15,Huazhong University of Science and Technology'...,0.833333,# Huazhong University of Science and Technolog...,8.0
7,18,Huazhong University of Science and Technology'...,0.833333,# Huazhong University of Science and Technolog...,8.0
8,11,HuaZhong University of Science and Technology'...,0.666667,# HuaZhong University of Science and Technolog...,8.0
9,9,Huazhong University of Science and Technology'...,0.555556,# Huazhong University of Science and Technolog...,7.5


In [15]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 2. LLM tokens: 9167
